# Rasterio plotting of Landsat-8 scenes

In this notebook, we will download bands of a Landsat-8 scene, visualize them with [rasterio's plotting module]( https://rasterio.readthedocs.io/en/latest/topics/plotting.html), and write an RGB image as rendered GeoTIFF.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.plot import show
import requests
from skimage import exposure

In [ ]:
%matplotlib inline

## Download and read bands

In [ ]:
landsat_url_suffixes = {'red': 'B4', 'green': 'B3', 'blue': 'B2', 'qa': 'BQA'}
landsat_url_prefix = 'http://landsat-pds.s3.amazonaws.com/c1/L8/008/067/LC08_L1TP_008067_20190405_20190405_01_RT/LC08_L1TP_008067_20190405_20190405_01_RT_'
landsat_urls = {k: f'{landsat_url_prefix}{v}.TIF' for k, v in landsat_url_suffixes.items()}

In [ ]:
def get_bands(band_urls, data_path='data', file_format='tif'):
    """Download and cache spectral bands of a satellite image
    
    Parameters
    ----------
    band_urls : dict
        URLs of individual bands: {<band_name>: <url>}
    data_path : string (optional)
        Location to save the data
    file_format : string (optional)
        File format of band

    Returns
    -------
    bands : dict
        Band arrays and the profile
        
    """
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    bands = {}

    for k, v in band_urls.items():
        print(os.path.basename(v))

        band_path = os.path.join(data_path, '{}.{}'.format(k, file_format))
        if not os.path.exists(band_path):
            print('Downloading...')
            r = requests.get(v)
            with open(band_path, 'wb') as f:
                f.write(r.content)
        else:
            print('Already downloaded...')

        with rasterio.open(band_path) as src:
            print('Reading...\n')
            bands[k] = src.read(1)
            if 'profile' not in bands:
                bands['profile'] = src.profile

    return bands

In [ ]:
landsat_bands = get_bands(landsat_urls, data_path='data', file_format='tif')

## Plot individual bands

Use `rasterio.plot.show()` to plot individual bands. The `transform` argument changes the image extent to the spatial bounds of the image.

In [ ]:
def plot_rgb_bands(bands):
    fig, (axr, axg, axb) = plt.subplots(1, 3, figsize=(21, 7))
    transform = bands['profile']['transform']
    show(bands['red'], ax=axr, cmap='Reds', title='Red band', transform=transform, vmax=np.percentile(bands['red'], 95))
    show(bands['green'], ax=axg, cmap='Greens', title='Green band', transform=transform, vmax=np.percentile(bands['red'], 95))
    show(bands['blue'], ax=axb, cmap='Blues', title='Blue band', transform=transform, vmax=np.percentile(bands['red'], 95))
    plt.show()

In [ ]:
plot_rgb_bands(landsat_bands)

## Create RGB stack

In [ ]:
def create_rgb_stack(bands, method='rescaling', percentile=2, clip_limit=0.03):
    """Create RGB stack from RGB bands

    Parameters
    ----------
    bands : dict
        Band arrays in {<band_name>: <array>} format, including 'red',
        'green', 'blue', and 'qa' (optional) keys
    method : string (optional)
        Method for modifying the band intensities. 'rescaling' stretches
        or shrinks the intensity range. 'clahe' applies Contrast Limited
        Adaptive Histogram Equalization, which is an algorithm for
        local contrast enhancement.
    percentile : int (optional)
        Shorthand for percentile range to compute (from percentile to
        100 - percentile) for intensity rescaling. Required when
        method='rescaling'.
    clip_limit : float (optional)
        Clipping limit. Required when method='clahe'.

    Returns
    -------
    ndarray
        RGB array (shape=(3, height, width), dtype='uint8')

    """
    modified_bands = []
    for band in [bands['red'], bands['green'], bands['blue']]:

        if method == 'rescaling':

            # Calculate percentiles, excluding fill pixels
            try:
                fill_mask = bands['qa'] == 1
                masked_band = np.ma.masked_where(fill_mask, band)
                masked_band = np.ma.filled(masked_band.astype('float'), np.nan)
            except KeyError:
                masked_band = band
            vmin, vmax = np.nanpercentile(masked_band,
                                          (percentile, 100 - percentile))

            # Rescale to percentile range
            modified_band = exposure.rescale_intensity(
                band, in_range=(vmin, vmax), out_range='uint8')
            modified_band = modified_band.astype(np.uint8)

        elif method == 'clahe':

            # Apply histogram equalization
            modified_band = exposure.equalize_adapthist(
                band, clip_limit=clip_limit)
            modified_band = (modified_band * 255).astype(np.uint8)

        modified_bands.append(modified_band)

    return np.stack(modified_bands)

In [ ]:
landsat_bands['rgb'] = create_rgb_stack(landsat_bands, method='clahe')

## Plot RGB image

In [ ]:
def plot_rgb_image(bands):
    plt.figure(figsize=(10, 10))
    show(bands['rgb'], transform=bands['profile']['transform'])

In [ ]:
plot_rgb_image(landsat_bands)

## Write RGB image

Write the RGB image as GeoTIFF and set 'RGB' color interpretation.

In [ ]:
def write_rgb_image(bands, data_path='data'):
    profile = bands['profile']
    profile.update(driver='GTiff', dtype=rasterio.uint8, count=3, photometric='RGB')
    rgb_path = os.path.join(data_path, 'rgb.tif')
    with rasterio.open(rgb_path, 'w', **profile) as dst:
        for i, band in enumerate(bands['rgb']):
            dst.write_band(i + 1, band)

In [ ]:
write_rgb_image(landsat_bands, data_path='data')